In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
# import yaml
# import re

# import pyarrow as pa
# import pyarrow.parquet as pq

# import gbd_mapping as gm
from vivarium import InteractiveContext
import scipy

# from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
# from get_draws.api import get_draws

# import vivarium_helpers as vh
# import vivarium_helpers.id_helper as idh
# from vivarium_helpers.vph_output.operations import VPHOperator
# from vivarium_helpers.vph_output import cleaning

!date
!whoami
!pwd

Wed Oct 22 12:33:07 PDT 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/verification_and_validation


In [2]:
! pip list | grep vivarium

vivarium                  3.5.5
vivarium_build_utils      2.0.11
vivarium_cluster_tools    2.1.21
vivarium_csu_alzheimers   0.1.dev49+gc7315b070 /mnt/share/code/ndbs/vivarium_csu_alzheimers
vivarium_dependencies     1.0.1
vivarium_public_health    4.3.12


In [3]:
! pip freeze | grep vivarium

vivarium==3.5.5
vivarium_build_utils==2.0.11
vivarium_cluster_tools==2.1.21
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@c7315b0703d0e097dce6e7600f2ac2846d4a4e31#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_public_health==4.3.12


# Automatically find the path to the model spec file

Code from [Vivarium Research documentation of interactive
simulations](https://vivarium-research.readthedocs.io/en/latest/model_design/designing_vivarium_model/interactive_sim/index.html).

See also the [interactive simulation tutorial in the Vivarium docs](https://vivarium.readthedocs.io/en/latest/tutorials/running_a_simulation/interactive.html).

In [4]:
import vivarium_csu_alzheimers
# model_specifications/model_spec.yaml is the typical location within the engineering repo for the
# default spec to live. Ask the engineers if you can't find it.
path = Path(vivarium_csu_alzheimers.__file__.replace('__init__.py', 'model_specifications/model_spec.yaml'))
print(path)

!cat $path

/mnt/share/code/ndbs/vivarium_csu_alzheimers/src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml
components:
    vivarium_csu_alzheimers:
      components:
          - ResultsStratifier()
          - NewSimulantsObserver()
          - AlzheimersIncidence()
          - AlzheimersPopulation('population.scaling_factor')
          - Alzheimers()
          - Testing()
          - BaselineTestingObserver()
          - BBBMTestingObserver()
          - Treatment()
          - TreatmentObserver()
          - TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')
            
    vivarium_public_health:
        population:
            - Mortality()
        results:
            - DisabilityObserver()
            - MortalityObserver()
            - DiseaseObserver('alzheimers_disease_and_other_dementias')

configuration:
    input_data:
        input_draw_number: 0
        artifact_path: '/mnt/team/simulation_

# Customize the model specification by deleting observers

In [5]:
from vivarium.framework.configuration import build_model_specification
custom_model_specification = build_model_specification(path)

# Delete mortality, morbidity and disease observers
print(custom_model_specification.components.vivarium_public_health.results)
del custom_model_specification.components.vivarium_public_health.results

# Filter out components with 'Observer' or 'Stratifier' in the name
observers = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if 'Observer' in c or 'Stratifier' in c
]
print(observers)
custom_model_specification.components.vivarium_csu_alzheimers.components = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if c not in observers
]

# Set scenario to intervention instead of baseline
custom_model_specification.configuration.intervention.scenario = 'bbbm_testing_and_treatment'

custom_model_specification.components.vivarium_csu_alzheimers.components


['DisabilityObserver()', 'MortalityObserver()', "DiseaseObserver('alzheimers_disease_and_other_dementias')"]
['ResultsStratifier()', 'NewSimulantsObserver()', 'BaselineTestingObserver()', 'BBBMTestingObserver()', 'TreatmentObserver()']


['AlzheimersIncidence()',
 "AlzheimersPopulation('population.scaling_factor')",
 'Alzheimers()',
 'Testing()',
 'Treatment()',
 "TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')"]

# Here's an alternative way to get a model spec and create a sim

In [6]:
location = "United States of America"
project_directory = Path("/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/")
model_spec_path = (
    project_directory /
    "results/model7.0/model_spec/2025_10_09_17_56_46/model_specification.yaml"
)
artifact_path = (
    project_directory /
    f"artifacts/model7.0/{location.lower().replace(' ', '_')}.hdf"
)

sim1 = InteractiveContext(model_spec_path, setup=False)
sim1.configuration.update(
    {
        'input_data': {'artifact_path': artifact_path},
        'intervention': {'scenario': 'bbbm_testing_and_treatment'},
    }
)
sim1.setup()

/ihme/homes/ndbs/miniconda3/envs/vivarium_csu_alzheimers_simulation/lib/python3.11/site-packages/vivarium_public_health/plugins/parser.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


2025-10-22 12:34:22.699 | INFO     | simulation_1-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model7.0/united_states_of_america.hdf.
2025-10-22 12:34:22.701 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-10-22 12:34:22.701 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.
2025-10-22 12:34:25.241 | INFO     | simulation_1-results_context:129 - The following stratifications are registered but not used by any observers: 
['current_year']
2025-10-22 12:34:25.378 | WARNING  | simulation_1-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-10-22 12:34:25.380 | WARNING  | simulation_1-resource_manager:176 - Resource stream.treatment_initial_states is not p

# Create a sim using custom model specification

In [7]:
sim = InteractiveContext(custom_model_specification)

2025-10-22 12:34:29.936 | INFO     | simulation_2-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model7.0/united_states_of_america.hdf.
2025-10-22 12:34:29.940 | INFO     | simulation_2-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-10-22 12:34:29.941 | INFO     | simulation_2-artifact_manager:81 - Artifact additional filter terms are None.
2025-10-22 12:34:31.876 | WARNING  | simulation_2-values_manager:55 - Unsourced pipelines: ['all_causes.disability_weight']
2025-10-22 12:34:31.879 | WARNING  | simulation_2-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-10-22 12:34:31.880 | WARNING  | simulation_2-resource_manager:176 - Resource stream.treatment_initial_states is not produced by any component but is needed to 

In [8]:
pop = sim.get_population()
pop

,tracked,exit_time,age,sex,location,alive,entrance_time,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_count,waning_effect_short_event_time,no_effect_after_long_event_count,no_effect_after_long_event_time,no_effect_after_short_event_count,no_effect_after_short_event_time,no_effect_never_treated_event_time,no_effect_never_treated_event_count
0,True,NaT,90.089181,Male,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
1,True,NaT,92.196689,Male,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
2,True,NaT,71.338013,Female,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
3,True,NaT,83.122725,Male,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
4,True,NaT,87.738486,Female,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,True,NaT,79.392384,Female,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
19996,True,NaT,85.789441,Female,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
19997,True,NaT,85.792091,Female,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
19998,True,NaT,75.836127,Female,United States of America,alive,2024-07-03,not_dead,0.0,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0


# Print list of columns and list of pipeline values

In [9]:
pop.columns

Index(['tracked', 'exit_time', 'age', 'sex', 'location', 'alive',
       'entrance_time', 'cause_of_death', 'years_of_life_lost',
       'alzheimers_disease_and_other_dementias', 'treatment',
       'bbbm_entrance_time',
       'alzheimers_blood_based_biomarker_state_event_time',
       'alzheimers_blood_based_biomarker_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_time',
       'alzheimers_disease_state_event_count',
       'alzheimers_disease_state_event_time', 'bbbm_test_result',
       'bbbm_test_ever_eligible', 'testing_propensity', 'bbbm_test_date',
       'testing_state', 'treatment_propensity',
       'susceptible_to_treatment_event_time',
       'susceptible_to_treatment_event_count', 'positive_test_event_time',
       'positive_test_event_count', 'start_treatment_event_time',
       'start_treatment_event_count', 'full_effect_long_event_count',
       'full_effect_long_event_time',

In [10]:
sim.list_values()

['simulant_step_size',
 'cause_specific_mortality_rate',
 'alzheimers_blood_based_biomarker_state.dwell_time',
 'alzheimers_blood_based_biomarker_state.disability_weight',
 'all_causes.disability_weight',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate.paf',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate',
 'mortality_rate',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate',
 'alzheimers_mild_cognitive_impairment_state.dwell_time',
 'alzheimers_mild_cognitive_impairment_state.disability_weight',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate.paf',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate.paf',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_s

In [11]:
[v for v in sim.list_values() if 'rate' in v]

['cause_specific_mortality_rate',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate.paf',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate',
 'mortality_rate',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate.paf',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate.paf',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate',
 'alzheimers_disease_state.excess_mortality_rate.paf',
 'alzheimers_disease_state.excess_mortality_rate',
 'start_treatment.excess_mortality_rate.paf',
 'start_treatment.excess_mortality_rate',
 'full_effect_long.excess_mortality_rate.paf',
 'full_effect_long.excess_mort

In [12]:
pop['alzheimers_disease_and_other_dementias']

0            alzheimers_blood_based_biomarker_state
1                          alzheimers_disease_state
2            alzheimers_blood_based_biomarker_state
3        alzheimers_mild_cognitive_impairment_state
4            alzheimers_blood_based_biomarker_state
                            ...                    
19995        alzheimers_blood_based_biomarker_state
19996        alzheimers_blood_based_biomarker_state
19997        alzheimers_blood_based_biomarker_state
19998                      alzheimers_disease_state
19999                      alzheimers_disease_state
Name: alzheimers_disease_and_other_dementias, Length: 20000, dtype: object

In [13]:
s = sim.get_value('alzheimers_mild_cognitive_impairment_state.dwell_time')(pop.index)
s.loc[s>0]

Series([], Name: alzheimers_mild_cognitive_impairment_state.dwell_time, dtype: float64)

In [14]:
s = sim.get_value('alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf')(pop.index)
s.loc[s>0]

Series([], Name: 0, dtype: float64)

# Check dwell times for treatment states

Looks good.

In [15]:
[v for v in sim.list_values() if 'dwell_time' in v]

['alzheimers_blood_based_biomarker_state.dwell_time',
 'alzheimers_mild_cognitive_impairment_state.dwell_time',
 'alzheimers_disease_state.dwell_time',
 'start_treatment.dwell_time',
 'full_effect_long.dwell_time',
 'full_effect_short.dwell_time',
 'waning_effect_long.dwell_time',
 'waning_effect_short.dwell_time',
 'no_effect_after_long.dwell_time',
 'no_effect_after_short.dwell_time',
 'no_effect_never_treated.dwell_time']

In [ ]:
# 1 time step = 182 days
sim.get_value('start_treatment.dwell_time')(pop.index).unique()

array([182.])

In [18]:
sim.get_value('full_effect_long.dwell_time')(pop.index).unique()

array([1820.])

In [19]:
sim.get_value('full_effect_short.dwell_time')(pop.index).unique()

array([182.])

In [ ]:
# Divide by 2 * 182 to convert to years
sim.get_value('waning_effect_long.dwell_time')(pop.index).unique() / (2 * 182)

array([9.])

In [24]:
# Divide by 2 * 182 to convert to years
sim.get_value('waning_effect_short.dwell_time')(pop.index).unique() / (2 * 182)

array([2.5])

In [ ]:
sim.get_value('no_effect_after_long.dwell_time')(pop.index).unique()

array([0.])

# Advance the population to a time when some people have treatment

In [26]:
sim.run_until(pd.Timestamp('2040-12-31'))

2025-10-22 12:42:36.330 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2025-01-01 00:00:00
2025-10-22 12:42:40.653 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2025-07-02 00:00:00
2025-10-22 12:42:45.315 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2025-12-31 00:00:00
2025-10-22 12:42:50.196 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2026-07-01 00:00:00
2025-10-22 12:42:55.123 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2026-12-30 00:00:00
2025-10-22 12:43:00.133 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2027-06-30 00:00:00
2025-10-22 12:43:05.278 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2027-12-29 00:00:00
2025-10-22 12:43:10.861 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2028-06-28 00:00:00
2025-10-22 12:43:16.171 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2028-12-27 00:00:00
2025-10-22 12:43:21.128 | INFO     | simulation_2 - vivarium.fra

In [27]:
pop = sim.get_population()
pop

,tracked,exit_time,age,sex,location,alive,entrance_time,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_count,waning_effect_short_event_time,no_effect_after_long_event_count,no_effect_after_long_event_time,no_effect_after_short_event_count,no_effect_after_short_event_time,no_effect_never_treated_event_time,no_effect_never_treated_event_count
0,True,2032-12-22,97.563514,Male,United States of America,dead,2024-07-03,other_causes,8.215451,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
1,True,2028-12-27,95.684711,Male,United States of America,dead,2024-07-03,other_causes,8.402510,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
2,True,NaT,87.781544,Female,United States of America,alive,2024-07-03,not_dead,0.000000,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
3,True,2031-06-25,89.102191,Male,United States of America,dead,2024-07-03,other_causes,9.242844,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
4,True,2025-07-02,87.738486,Female,United States of America,dead,2024-07-03,other_causes,9.744761,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78250,True,NaT,89.492027,Male,NaN,alive,2040-12-12,not_dead,0.000000,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
78251,True,NaT,86.141478,Male,NaN,alive,2040-12-12,not_dead,0.000000,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
78252,True,NaT,88.860750,Male,NaN,alive,2040-12-12,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0
78253,True,NaT,87.154364,Male,NaN,alive,2040-12-12,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,NaT,0


In [29]:
sim.get_value('treatment.exposure')(pop.index).unique()

array(['susceptible_to_treatment', 'no_effect_never_treated',
       'start_treatment', 'waning_effect_long', 'full_effect_long',
       'no_effect_after_short', 'waning_effect_short',
       'full_effect_short'], dtype=object)

In [ ]:
# Why are some relative risks > 1.0?
# I think it only happens for dead simulants -- we'll check later
sim.get_value(
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk')(pop.index).unique()

array([1.        , 0.8425053 , 0.52751591, 0.55376502, 0.76375795,
       0.6850106 , 0.71125972, 0.58001414, 0.79000707, 0.63251237,
       0.73750884, 0.81625619, 0.65876149, 0.60626326, 0.90550318,
       0.81100636, 2.13396182, 2.03946501, 1.85047137, 1.75597455,
       0.62201273, 1.56698091, 0.71650954])

In [31]:
sim.current_time

Timestamp('2041-06-12 00:00:00')

# Define a function to get columns and pipeline values we want from the population

In [32]:
# List columns we want to keep
columns = [
    'age', 'alive', 'bbbm_test_result', 'treatment',
    'alzheimers_disease_and_other_dementias',
    'bbbm_entrance_time',
]
# Rename some columns with shorter names if desired
columns_to_short_names = {c: c for c in columns} | {
    'alzheimers_disease_and_other_dementias': 'ad_state',
}
# Simultaneously list pipeline values we want and map them to shorter
# names
values_to_short_names = {
    'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate':
        'mci_inc_rate',
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk':
        'relative_risk',
}

def get_pop_columns(
        sim,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Get the specified subset of population columns concatenated with
    the specified pipeline values, with columns for pipeline values
    renamed according to the names in the provided "short names"
    dictionary.
    """
    pop = sim.get_population()
    pipeline_values = [
        sim.get_value(v)(pop.index).rename(short_name)
        for v, short_name in values_to_short_names.items()]
    pop_columns = (
        pop[columns_to_short_names.keys()]
        .rename(columns=columns_to_short_names)
        .join(pipeline_values)
        # Add current time to the dataframe
        .assign(current_time=sim.current_time)
    )
    return pop_columns

def take_steps(
        sim,
        n,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Take n simulation steps, recording the specified population
    columns and values at each step, and return a list of the population
    tables at each step. The resulting list will have n+1 population
    tables.
    """
    population_trace = [
        get_pop_columns(sim, columns_to_short_names, values_to_short_names)]
    for _ in range(n):
        sim.step()
        population_trace.append(get_pop_columns(
            sim, columns_to_short_names, values_to_short_names))
    return population_trace

pop1 = get_pop_columns(sim)
pop1

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-08-03 07:39:38.969251390,0.056241,1.0,2041-06-12
1,95.684711,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-07-19 05:22:18.080798424,0.055600,1.0,2041-06-12
2,87.781544,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.056446,1.0,2041-06-12
3,89.102191,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2022-03-11 15:55:47.288939655,0.056499,1.0,2041-06-12
4,87.738486,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2020-10-12 08:00:15.290018928,0.057385,1.0,2041-06-12
...,...,...,...,...,...,...,...,...,...
78250,89.492027,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78251,86.141478,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78252,88.860750,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78253,87.154364,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12


# See what the relative risks look like when they're not 1.0

In [36]:
pop1.query("relative_risk != 1.0")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
146,87.648047,alive,positive,waning_effect_long,alzheimers_disease_state,2024-10-07 15:06:10.903031519,0.046120,0.842505,2041-06-12
360,86.536714,dead,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2023-09-25 20:09:56.556607170,0.046734,0.842505,2041-06-12
618,77.842615,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2021-09-10 17:31:50.672850193,0.047869,0.842505,2041-06-12
1182,81.964068,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2021-06-23 11:57:18.487730747,0.047984,0.842505,2041-06-12
1191,73.798045,alive,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2023-12-12 13:01:32.064051077,0.029184,0.527516,2041-06-12
...,...,...,...,...,...,...,...,...,...
74036,77.998849,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12
74037,78.233413,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12
74039,77.707071,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12
74041,80.301554,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12


# Test the take_steps function

In [37]:
pop_trace = take_steps(sim, 4)
len(pop_trace)

2025-10-22 12:59:18.698 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2041-06-12 00:00:00
2025-10-22 12:59:28.506 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2041-12-11 00:00:00
2025-10-22 12:59:38.833 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2042-06-11 00:00:00
2025-10-22 12:59:48.975 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2042-12-10 00:00:00


5

In [38]:
pop_trace[0]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-08-03 07:39:38.969251390,0.056241,1.0,2041-06-12
1,95.684711,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-07-19 05:22:18.080798424,0.055600,1.0,2041-06-12
2,87.781544,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.056446,1.0,2041-06-12
3,89.102191,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2022-03-11 15:55:47.288939655,0.056499,1.0,2041-06-12
4,87.738486,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2020-10-12 08:00:15.290018928,0.057385,1.0,2041-06-12
...,...,...,...,...,...,...,...,...,...
78250,89.492027,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78251,86.141478,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78252,88.860750,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78253,87.154364,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12


In [39]:
pop_trace[1]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-08-03 07:39:38.969251390,0.056565,1.0,2041-12-11
1,95.684711,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-07-19 05:22:18.080798424,0.055937,1.0,2041-12-11
2,88.279833,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.056765,1.0,2041-12-11
3,89.102191,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2022-03-11 15:55:47.288939655,0.056817,1.0,2041-12-11
4,87.738486,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2020-10-12 08:00:15.290018928,0.057686,1.0,2041-12-11
...,...,...,...,...,...,...,...,...,...
80211,85.846761,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11
80212,89.191160,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11
80213,90.204980,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11
80214,87.646724,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11


In [40]:
pop_trace[2]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-08-03 07:39:38.969251390,0.056882,1.0,2042-06-11
1,95.684711,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-07-19 05:22:18.080798424,0.056266,1.0,2042-06-11
2,88.778122,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.057078,1.0,2042-06-11
3,89.102191,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2022-03-11 15:55:47.288939655,0.057130,1.0,2042-06-11
4,87.738486,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2020-10-12 08:00:15.290018928,0.057982,1.0,2042-06-11
...,...,...,...,...,...,...,...,...,...
82148,89.904601,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-12-11 00:00:00.000000000,0.025287,1.0,2042-06-11
82149,86.251081,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-12-11 00:00:00.000000000,0.025287,1.0,2042-06-11
82150,87.606108,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2041-12-11 00:00:00.000000000,0.025287,1.0,2042-06-11
82151,86.623222,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-12-11 00:00:00.000000000,0.025287,1.0,2042-06-11


# Define Weibull distribution and hazard function

We'll use this to check the MCI incidence rate.

In [41]:
λ=6.76
k=1.22

weibull_dist = scipy.stats.weibull_min(k, scale=λ)
weibull_dist.cdf([5, 10])

array([0.49950771, 0.8005858 ])

In [42]:
def hazard(t, dist):
    # hazard = (probability density) / (survival function)
    return dist.pdf(t) / dist.sf(t)

def hazard_with_logs(t, dist):
    # Take logs first to avoid underflow
    return np.exp(dist.logpdf(t) - dist.logsf(t))

mci_hazard_func = lambda t: hazard_with_logs(t, weibull_dist)

In [43]:
DAYS_PER_YEAR = 365.25

def years_in_bbbm(current_time, bbbm_entrance_time):
    return (current_time - bbbm_entrance_time).dt.days / DAYS_PER_YEAR

def mci_incidence_hazard(pop):
    hazard = mci_hazard_func(
        years_in_bbbm(pop['current_time'], pop['bbbm_entrance_time']))
    return pd.Series(hazard, index=pop.index, name='mci_hazard')

mci_incidence_hazard(pop_trace[1])

0        0.227465
1        0.224938
2        0.228274
3        0.228483
4        0.231976
           ...   
80211    0.101688
80212    0.101688
80213    0.101688
80214    0.101688
80215    0.101688
Name: mci_hazard, Length: 80216, dtype: float64

# Check that calculated Weibull hazard is increasing over time

Yes, looks good.

In [44]:
pd.concat([mci_incidence_hazard(pop) * pop['relative_risk'] for pop in pop_trace], axis=1)

,0,1,2,3,4
0,0.226163,0.227465,0.228740,0.229991,0.231218
1,0.223583,0.224938,0.226264,0.227564,0.228837
2,0.226989,0.228274,0.229533,0.230769,0.231982
3,0.227202,0.228483,0.229739,0.230971,0.232180
4,0.230763,0.231976,0.233166,0.234336,0.235485
...,...,...,...,...,...
86084,NaN,NaN,NaN,NaN,0.101688
86085,NaN,NaN,NaN,NaN,0.101688
86086,NaN,NaN,NaN,NaN,0.101688
86087,NaN,NaN,NaN,NaN,0.101688


# Add the calculated MCI incidence hazard to each population in the trace

In [45]:
pops = [pop.assign(mci_hazard=mci_incidence_hazard(pop)) for pop in pop_trace]
pops[0]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
0,97.563514,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-08-03 07:39:38.969251390,0.056241,1.0,2041-06-12,0.226163
1,95.684711,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-07-19 05:22:18.080798424,0.055600,1.0,2041-06-12,0.223583
2,87.781544,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.056446,1.0,2041-06-12,0.226989
3,89.102191,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2022-03-11 15:55:47.288939655,0.056499,1.0,2041-06-12,0.227202
4,87.738486,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2020-10-12 08:00:15.290018928,0.057385,1.0,2041-06-12,0.230763
...,...,...,...,...,...,...,...,...,...,...
78250,89.492027,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688
78251,86.141478,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688
78252,88.860750,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688
78253,87.154364,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688


# It looks like the calculated Weibull hazard does not match the incidence rate in the sim...

The `mci_inc_rate` column should be the _probability_ of transitioning,
i.e., the hazard times the step size, which is about 0.5. Instead of
being about half the hazard, it looks like it's about 1/4 the hazard...

In [46]:
pops[0].query("alive=='alive'")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
2,87.781544,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.056446,1.0,2041-06-12,0.226989
6,77.479474,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2020-10-04 16:22:15.797531118,0.057398,1.0,2041-06-12,0.230816
15,87.730920,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2020-03-19 12:02:19.609739255,0.057727,1.0,2041-06-12,0.232140
32,100.130892,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-12-05 11:05:19.249276937,0.057898,1.0,2041-06-12,0.232828
35,79.646194,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-10-24 15:50:37.563584386,0.055416,1.0,2041-06-12,0.222849
...,...,...,...,...,...,...,...,...,...,...
78250,89.492027,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688
78251,86.141478,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688
78252,88.860750,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688
78253,87.154364,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12,0.101688


In [47]:
pops[1].query("alive=='alive'")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
2,88.279833,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.056765,1.0,2041-12-11,0.228274
6,77.977763,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2020-10-04 16:22:15.797531118,0.057699,1.0,2041-12-11,0.232028
15,88.229209,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2020-03-19 12:02:19.609739255,0.058022,1.0,2041-12-11,0.233328
42,89.394093,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2024-12-21 15:54:29.991097765,0.054951,1.0,2041-12-11,0.220979
46,89.474901,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-08-01 09:47:13.645188902,0.055912,1.0,2041-12-11,0.224842
...,...,...,...,...,...,...,...,...,...,...
80211,85.846761,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11,0.101688
80212,89.191160,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11,0.101688
80213,90.204980,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11,0.101688
80214,87.646724,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2041-06-12 00:00:00.000000000,0.025287,1.0,2041-12-11,0.101688


# Check where the relative risk is not 1.0

The RR should always be < 1.0. For simulants in one of the two "full
effect" states, the RR should always be equal to the uniform draw from
[0.4, 0.6] -- we could check the specific draw in the artifact if we
wanted. For simulants in one of the two "waning effect" states, the RR
should be somewhere between this uniform draw and 1.0.

Some simulants are ending up with an RR **greater** than 1.0, which
seams weird. It looks like all such simulants are dead and in one of the
"waning effect" states, so my guess is that when you compute the
pipeline value for these simulants, i'ts just using the linear function
from the waning state, which will end up being > 1.0 because the dead
simulant has been "in that state" longer than the duration of the state.

In [48]:
pops[1].query("relative_risk < 1")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
146,88.146335,alive,positive,waning_effect_long,alzheimers_disease_state,2024-10-07 15:06:10.903031519,0.047866,0.868754,2041-12-11,0.221564
360,86.536714,dead,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2023-09-25 20:09:56.556607170,0.048486,0.868754,2041-12-11,0.224435
618,78.340903,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2021-09-10 17:31:50.672850193,0.049631,0.868754,2041-12-11,0.229739
1182,82.462357,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2021-06-23 11:57:18.487730747,0.049748,0.868754,2041-12-11,0.230276
1191,74.296334,alive,positive,full_effect_long,alzheimers_mild_cognitive_impairment_state,2023-12-12 13:01:32.064051077,0.029365,0.527516,2041-12-11,0.223853
...,...,...,...,...,...,...,...,...,...,...
75870,80.321004,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2040-06-13 00:00:00.000000000,0.016986,0.527516,2041-12-11,0.129490
75898,77.215240,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2040-06-13 00:00:00.000000000,0.016986,0.527516,2041-12-11,0.129490
75913,78.621123,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2040-06-13 00:00:00.000000000,0.016986,0.527516,2041-12-11,0.129490
75920,78.926684,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2040-06-13 00:00:00.000000000,0.016986,0.527516,2041-12-11,0.129490


In [49]:
pops[1].query("relative_risk > 1 ")

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
42230,77.747717,dead,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2031-06-25,0.110100,2.228459,2041-12-11,0.198682
43037,81.888714,dead,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2031-12-24,0.053498,1.094497,2041-12-11,0.196561
44102,78.700234,dead,positive,waning_effect_short,alzheimers_mild_cognitive_impairment_state,2031-12-24,0.104305,2.133962,2041-12-11,0.196561
46696,81.039609,dead,positive,waning_effect_short,alzheimers_mild_cognitive_impairment_state,2032-12-22,0.092889,1.944968,2041-12-11,0.192057
49427,78.998983,dead,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2033-06-22,0.087272,1.850471,2041-12-11,0.189657
52378,80.835852,dead,positive,waning_effect_short,alzheimers_mild_cognitive_impairment_state,2034-06-21,0.076230,1.661478,2041-12-11,0.184506


# Check the ratio of the computed Weibull hazard to the MCI incidene rate in the sim

It looks like it's *usually* equal to 4.021402, whereas it should be
around 2.0. The engineers and Zeb confirmed that this is a bug where the
hazard function is getting multiplied by the step size of $\text{182
days} \approx \text{0.5 years}$ *twice* instead of once, so the
probability is about half as big as it should be.

In [166]:
(pops[1]['mci_hazard'] / pops[1]['mci_inc_rate']).describe()

count    99710.000000
mean         4.085100
std          0.450777
min          1.267211
25%          4.021402
50%          4.021402
75%          4.021402
max          7.623282
dtype: float64

In [ ]:
# Hmm, the mean goes *up* when we filter to alive simulants
# Oh, maybe this makes sense because simulants with treatment will have
# smaller hazards than untreated simulants, making this ratio bigger
# than 4
temp = pops[1].query("alive=='alive'")
(temp['mci_hazard'] / temp['mci_inc_rate']).describe()

count    40216.000000
mean         4.115768
std          0.546666
min          4.021262
25%          4.021402
50%          4.021402
75%          4.021402
max          7.623282
dtype: float64

# Check the treatment effect size

Even though the transion probabilities are wrong, the **hazard ratio**
should still validate.

In [52]:
# Filter to simulants in BBBM state
bbbm = pop1.query("alive=='alive' and ad_state=='alzheimers_blood_based_biomarker_state'")
bbbm

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
211,87.747843,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-09-14 17:12:21.750244168,0.056810,1.0,2041-06-12
245,92.261141,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2020-01-31 11:43:50.206468779,0.057805,1.0,2041-06-12
248,91.234793,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-07-25 13:41:58.103387520,0.056257,1.0,2041-06-12
252,89.514547,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2020-01-06 01:33:15.902430171,0.057847,1.0,2041-06-12
288,91.759845,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-10-13 07:12:17.232513402,0.056761,1.0,2041-06-12
...,...,...,...,...,...,...,...,...,...
78245,87.164548,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78247,88.258366,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78248,89.691848,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78252,88.860750,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12


In [53]:
bbbm.treatment.unique()

array(['susceptible_to_treatment', 'no_effect_never_treated',
       'waning_effect_long', 'no_effect_after_short', 'full_effect_long',
       'start_treatment', 'waning_effect_short', 'full_effect_short'],
      dtype=object)

In [54]:
# Filter to simulants in BBBM state who are getting full treatment effect
bbbm_full_effect = bbbm.query("treatment in ['full_effect_long', 'full_effect_short']")
bbbm_full_effect

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
5176,75.795928,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2023-08-14 16:36:49.405812203,0.029304,0.527516,2041-06-12
7965,77.551035,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2024-03-27 13:28:24.447267865,0.029077,0.527516,2041-06-12
9395,77.403509,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2024-12-11 05:51:25.415066390,0.028809,0.527516,2041-06-12
11596,82.875669,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2024-09-19 09:13:57.791141696,0.028896,0.527516,2041-06-12
13589,82.832821,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2024-08-19 21:33:05.196899471,0.028928,0.527516,2041-06-12
...,...,...,...,...,...,...,...,...,...
74036,77.998849,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12
74037,78.233413,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12
74039,77.707071,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12
74041,80.301554,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12


In [55]:
# Filter to simulants in BBBM state with no treatment effect
bbbm_no_effect = bbbm.query(
    "treatment in ['susceptible_to_treatment', 'no_effect_never_treated', 'no_effect_after_long', 'no_effect_after_short']")
bbbm_no_effect

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
211,87.747843,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-09-14 17:12:21.750244168,0.056810,1.0,2041-06-12
245,92.261141,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2020-01-31 11:43:50.206468779,0.057805,1.0,2041-06-12
248,91.234793,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-07-25 13:41:58.103387520,0.056257,1.0,2041-06-12
252,89.514547,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2020-01-06 01:33:15.902430171,0.057847,1.0,2041-06-12
288,91.759845,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-10-13 07:12:17.232513402,0.056761,1.0,2041-06-12
...,...,...,...,...,...,...,...,...,...
78245,87.164548,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78247,88.258366,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78248,89.691848,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12
78252,88.860750,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2040-12-12 00:00:00.000000000,0.025287,1.0,2041-06-12


In [56]:
# Compute ratio of MCI incidence rates between the two groups
bbbm_full_effect.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.5340948973528488

In [57]:
# This should be a unique value that's close to the ratio above
# Yes, looks good!
bbbm_full_effect.relative_risk.unique()

array([0.52751591])

# Check hazard ratio in waning states

Hmm, the ratio of means for simulants in the `waning_effect_short` state
is close to the mean of the hazard ratios, but for simulants in the
`waning_effect_long` state, the ratio of means is larger than I would
expect -- closer to the maximum hazard ratio than to the mean. Could
this be due to not having any simulants who are near the end of the
`waning_effect_long` state?

I don't know if that makes sense, and I'm not sure what the right answer
should be...



In [107]:
bbbm_no_effect.mci_inc_rate.describe()

count    14188.000000
mean         0.040191
std          0.008130
min          0.025287
25%          0.034304
50%          0.041004
75%          0.046537
max          0.058617
Name: mci_inc_rate, dtype: float64

In [ ]:
bbbm_waning_long = bbbm.query("treatment == 'waning_effect_long'")
bbbm_waning_long.relative_risk.describe()

count    293.000000
mean       0.685727
std        0.097852
min        0.553765
25%        0.580014
50%        0.685011
75%        0.763758
max        0.842505
Name: relative_risk, dtype: float64

In [ ]:
# Hmm, this is closer to the maximum hazard ratio than to the mean...
# Maybe this is because we haven't run the sim long enough for anyone to
# fully go through this state yet?
bbbm_waning_long.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.8512023308605532

In [ ]:
# That's weird -- why do some simulants have an RR of 1.0 when they're
# still in the waning effect state?
bbbm_waning_short = bbbm.query("treatment == 'waning_effect_short'")
bbbm_waning_short.relative_risk.describe()

count    37.000000
mean      0.828884
std       0.137007
min       0.622013
25%       0.716510
50%       0.811006
75%       1.000000
max       1.000000
Name: relative_risk, dtype: float64

In [63]:
bbbm_waning_short.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.8332166155238557

# Look into RRs of 1.0 in the `waning_effect_short` state

It looks like this happens on the 4th time step of the state, out of 5
total steps. So maybe this is an off-by-one error?

In [64]:
bbbm_waning_short

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
31780,64.407065,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2028-06-28,0.046890,0.905503,2041-06-12
40941,71.418861,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2031-06-25,0.039641,0.811006,2041-06-12
42078,78.317031,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2031-06-25,0.044260,0.905503,2041-06-12
43037,81.888714,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2031-12-24,0.048330,1.000000,2041-06-12
51032,69.642543,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2033-12-21,0.045881,1.000000,2041-06-12
52005,77.184495,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2034-06-21,0.028109,0.622013,2041-06-12
53529,62.385902,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2034-12-20,0.031855,0.716510,2041-06-12
57506,79.378184,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2035-12-19,0.042855,1.000000,2041-06-12
57783,80.821839,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2035-12-19,0.038805,0.905503,2041-06-12
59515,81.940955,alive,positive,waning_effect_short,alzheimers_blood_based_biomarker_state,2036-06-18,0.030069,0.716510,2041-06-12


In [ ]:
# NOTE: Since I advanced the simulation further, the current time in the
# table below is in the future relative to the tables above. So some of
# the times will be in the future relative to the "current_time" in
# those tables.
sim.current_time

Timestamp('2043-06-10 00:00:00')

In [104]:
p = sim.get_population()
p.loc[bbbm_waning_short.index, [v for v in pop.columns if 'event_time' in v]]

,alzheimers_blood_based_biomarker_state_event_time,alzheimers_mild_cognitive_impairment_state_event_time,alzheimers_disease_state_event_time,susceptible_to_treatment_event_time,positive_test_event_time,start_treatment_event_time,full_effect_long_event_time,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_short_event_time,no_effect_after_long_event_time,no_effect_after_short_event_time,no_effect_never_treated_event_time
31780,2028-06-28,NaT,NaT,NaT,2038-12-15,2038-12-15,NaT,2039-06-15,NaT,2039-12-14,NaT,2042-06-11,NaT
40941,2031-06-25,NaT,NaT,NaT,2039-06-15,2039-06-15,NaT,2039-12-14,NaT,2040-06-13,NaT,2042-12-10,NaT
42078,2031-06-25,NaT,NaT,NaT,2038-12-15,2038-12-15,NaT,2039-06-15,NaT,2039-12-14,NaT,2042-06-11,NaT
43037,2031-12-24,NaT,NaT,NaT,2038-06-16,2038-06-16,NaT,2038-12-15,NaT,2039-06-15,NaT,NaT,NaT
51032,2033-12-21,NaT,NaT,NaT,2038-06-16,2038-06-16,NaT,2038-12-15,NaT,2039-06-15,NaT,2041-12-11,NaT
52005,2034-06-21,NaT,NaT,NaT,2040-06-13,2040-06-13,NaT,2040-12-12,NaT,2041-06-12,NaT,NaT,NaT
53529,2034-12-20,NaT,NaT,NaT,2039-12-14,2039-12-14,NaT,2040-06-13,NaT,2040-12-12,NaT,2043-06-10,NaT
57506,2035-12-19,NaT,NaT,NaT,2038-06-16,2038-06-16,NaT,2038-12-15,NaT,2039-06-15,NaT,2041-12-11,NaT
57783,2035-12-19,NaT,NaT,NaT,2038-12-15,2038-12-15,NaT,2039-06-15,NaT,2039-12-14,NaT,2042-06-11,NaT
59515,2036-06-18,2041-12-11,NaT,NaT,2039-12-14,2039-12-14,NaT,2040-06-13,NaT,2040-12-12,NaT,NaT,NaT


In [ ]:
# The simulants with an RR of 1.0 are on the 4th timestep of the waning state
(pd.Timestamp('2041-06-12') - pd.Timestamp('2039-06-15')).days / 182

4.0

In [105]:
p.loc[bbbm_waning_long.index, [v for v in pop.columns if 'event_time' in v]]

,alzheimers_blood_based_biomarker_state_event_time,alzheimers_mild_cognitive_impairment_state_event_time,alzheimers_disease_state_event_time,susceptible_to_treatment_event_time,positive_test_event_time,start_treatment_event_time,full_effect_long_event_time,full_effect_short_event_time,waning_effect_long_event_time,waning_effect_short_event_time,no_effect_after_long_event_time,no_effect_after_short_event_time,no_effect_never_treated_event_time
618,2024-07-03,NaT,NaT,NaT,2030-06-26,2030-06-26,2030-12-25,NaT,2035-12-19,NaT,NaT,NaT,NaT
1182,2024-07-03,NaT,NaT,NaT,2030-06-26,2030-06-26,2030-12-25,NaT,2035-12-19,NaT,NaT,NaT,NaT
1733,2024-07-03,NaT,NaT,NaT,2031-12-24,2031-12-24,2032-06-23,NaT,2037-06-17,NaT,NaT,NaT,NaT
2445,2024-07-03,NaT,NaT,NaT,2030-06-26,2030-06-26,2030-12-25,NaT,2035-12-19,NaT,NaT,NaT,NaT
3854,2024-07-03,NaT,NaT,NaT,2035-12-19,2035-12-19,2036-06-18,NaT,2041-06-12,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54894,2034-12-20,NaT,NaT,NaT,2035-12-19,2035-12-19,2036-06-18,NaT,2041-06-12,NaT,NaT,NaT,NaT
54937,2034-12-20,NaT,NaT,NaT,2035-12-19,2035-12-19,2036-06-18,NaT,2041-06-12,NaT,NaT,NaT,NaT
54980,2034-12-20,NaT,NaT,NaT,2035-12-19,2035-12-19,2036-06-18,NaT,2041-06-12,NaT,NaT,NaT,NaT
55054,2034-12-20,NaT,NaT,NaT,2035-12-19,2035-12-19,2036-06-18,NaT,2041-06-12,NaT,NaT,NaT,NaT


In [ ]:
# No simulants have reached the end of the waning_effect_long state yet
p.query("no_effect_after_long_event_time.notna()")

,tracked,exit_time,age,sex,location,alive,entrance_time,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_count,waning_effect_short_event_time,no_effect_after_long_event_count,no_effect_after_long_event_time,no_effect_after_short_event_count,no_effect_after_short_event_time,no_effect_never_treated_event_time,no_effect_never_treated_event_count


# Compute range of expected RRs in waning effect states

In [ ]:
# Range of expected RR values in 'waning_effect_short' state
RR = bbbm_full_effect.relative_risk.unique()[0]
s = 5
[RR + i * (1 - RR) / s for i in range(s+2)]

[0.5275159063282223,
 0.6220127250625779,
 0.7165095437969333,
 0.8110063625312889,
 0.9055031812656444,
 1.0,
 1.0944968187343556]

In [84]:
bbbm_waning_short.relative_risk.describe()

count    37.000000
mean      0.828884
std       0.137007
min       0.622013
25%       0.716510
50%       0.811006
75%       1.000000
max       1.000000
Name: relative_risk, dtype: float64

In [ ]:
# Range of expected RR values in 'waning_effect_long' state
s = 9 * 2
[RR + i * (1 - RR) / s for i in range(s+2)]

[0.5275159063282223,
 0.5537650226433211,
 0.5800141389584198,
 0.6062632552735185,
 0.6325123715886174,
 0.6587614879037161,
 0.6850106042188149,
 0.7112597205339136,
 0.7375088368490124,
 0.7637579531641112,
 0.7900070694792098,
 0.8162561857943087,
 0.8425053021094074,
 0.8687544184245062,
 0.895003534739605,
 0.9212526510547037,
 0.9475017673698025,
 0.9737508836849013,
 1.0,
 1.0262491163150989]

In [83]:
bbbm_waning_long.relative_risk.describe()

count    293.000000
mean       0.685727
std        0.097852
min        0.553765
25%        0.580014
50%        0.685011
75%        0.763758
max        0.842505
Name: relative_risk, dtype: float64

# Check treatment state durations

Looks good based on recorded event times, but it looks like I haven't
run the sim long enough yet to check the `waning_effect_long` state.

In [ ]:
p = sim.get_population()

In [97]:
completed_short_treatment = p.query("no_effect_after_short_event_time.notna()")
(
    completed_short_treatment['no_effect_after_short_event_time']
    - completed_short_treatment['waning_effect_short_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([2.5])

In [96]:
entered_waning_short_treatment = p.query("waning_effect_short_event_time.notna()")
(
    entered_waning_short_treatment['waning_effect_short_event_time']
    - entered_waning_short_treatment['full_effect_short_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([0.5])

In [ ]:
# I haven't run the sim long enough to observe anyone complete long
# treatment yet
completed_long_treatment = p.query("no_effect_after_long_event_time.notna()")
(
    completed_long_treatment['no_effect_after_long_event_time']
    - completed_long_treatment['waning_effect_long_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([], dtype=float64)

In [102]:
entered_waning_long_treatment = p.query("waning_effect_long_event_time.notna()")
(
    entered_waning_long_treatment['waning_effect_long_event_time']
    - entered_waning_long_treatment['full_effect_long_event_time']
).unique() / (2 * pd.Timedelta('182 days')) # Convert to years

array([5.])

# Compare calculated Weibull hazard to hazard rate value from pipeline

In [ ]:
# Among those with no treatment effect, the ratio should be the same for
# everyone.
# Hmm, why is there still some variation in this? Maybe rounding errors,
# somehow?
ratio = (mci_incidence_hazard(bbbm_no_effect) / bbbm_no_effect['mci_inc_rate']).rename('haz_to_prob_ratio')
ratio.describe()

count    14188.000000
mean         4.021401
std          0.000012
min          4.021264
25%          4.021402
50%          4.021402
75%          4.021402
max          4.021402
Name: haz_to_prob_ratio, dtype: float64

In [109]:
# Look into values that are smaller than expected
bbbm_no_effect.join(mci_incidence_hazard(bbbm_no_effect)).join(ratio).loc[ratio < 4.021402]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,haz_to_prob_ratio
211,87.747843,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-09-14 17:12:21.750244168,0.056810,1.0,2041-06-12,0.228455,4.021368
245,92.261141,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2020-01-31 11:43:50.206468779,0.057805,1.0,2041-06-12,0.232456,4.021344
248,91.234793,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-07-25 13:41:58.103387520,0.056257,1.0,2041-06-12,0.226228,4.021347
252,89.514547,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2020-01-06 01:33:15.902430171,0.057847,1.0,2041-06-12,0.232619,4.021297
288,91.759845,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-10-13 07:12:17.232513402,0.056761,1.0,2041-06-12,0.228253,4.021316
...,...,...,...,...,...,...,...,...,...,...,...
19638,86.467438,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2019-02-15 00:55:44.874820156,0.058367,1.0,2041-06-12,0.234711,4.021298
19747,75.957288,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-02-01 09:00:22.681006594,0.055226,1.0,2041-06-12,0.222083,4.021315
19856,81.488621,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2020-11-09 08:57:58.880481503,0.057338,1.0,2041-06-12,0.230574,4.021329
19908,97.681717,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2018-10-11 14:34:01.457554647,0.058565,1.0,2041-06-12,0.235510,4.021360


In [110]:
# The ratio should be larger than the value above for people getting treatment
ratio = (mci_incidence_hazard(bbbm) /  bbbm['mci_inc_rate']).rename('haz_to_prob_ratio')
bbbm.join(mci_incidence_hazard(bbbm)).join(ratio).loc[ratio > 5]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,haz_to_prob_ratio
1733,80.499556,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2021-04-06 01:14:28.370333713,0.043601,0.763758,2041-06-12,0.229568,5.265135
3854,81.384796,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2023-12-29 19:57:09.720606096,0.030618,0.553765,2041-06-12,0.222344,7.261887
5176,75.795928,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2023-08-14 16:36:49.405812203,0.029304,0.527516,2041-06-12,0.223387,7.623203
5697,85.719942,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2022-07-27 14:03:28.068011398,0.038534,0.685011,2041-06-12,0.226214,5.870492
7287,83.268200,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2022-01-27 10:27:33.838132487,0.031329,0.553765,2041-06-12,0.227507,7.261802
...,...,...,...,...,...,...,...,...,...,...,...
74036,77.998849,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12,0.129490,7.623282
74037,78.233413,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12,0.129490,7.623282
74039,77.707071,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12,0.129490,7.623282
74041,80.301554,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2039-12-14 00:00:00.000000000,0.016986,0.527516,2041-06-12,0.129490,7.623282
